# Functionality of this notebook: 

* calculate the ciCOH for normal and mild LFP data 

* stk is short for single target kluver

* load folder

## Import Packages

In [1]:
import os, sys
import scipy.io as sio
import glob
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
%matplotlib inline

In [2]:
# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# import util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import ciCoherence_overtime in connAnalyTool/synchronization_indices.ipynb
from connAnalyTool.synchronization_indices import ciCoherence_acrosstrials
from connAnalyTool.synchronization_indices import ciCoherence_overtime

importing Jupyter notebook from /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb
importing Jupyter notebook from /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/connAnalyTool/synchronization_indices.ipynb


## exp subfolders & code_corresfolder

In [3]:
_, _, pipelinefolder, _= exp_subfolders()

In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder, correparentfolder = code_corresfolder(os.getcwd(), nb_name)

## global parameters

In [6]:
eventphase = 'return'

freq = [17, 19]

animal = 'Pinky'

leadSTN_chns, leadGP_chns = [5,6], [6, 7] # start from 0

In [7]:
coli_target, coli_reachonset, coli_touch, coli_returnonset, coli_mouth = 0, 1, 2, 3, 4
cols = [coli_returnonset, coli_mouth]

In [8]:
tdur_min = 0.5
tdur_max = 1

## Input step

In [9]:
inputfolder = os.path.join(pipelinefolder, 'NHP_Pinky/0_dataPrep/SKT/m3_STKData_narrowfiltered17_19')

In [10]:
chnInf_folder = correparentfolder
chnInf_file = os.path.join(chnInf_folder, 'chn_brainArea_simCoord_M1ThaSMADBS.csv')

## save file setup

In [11]:
savefolder = corresfolder
savefilename =  'ciCOH_' + eventphase + '_freq[' + str(freq[0]) + ' ' + str(freq[1]) +'].pickle'
savefile_ciCOH = os.path.join(savefolder, savefilename)

## Extract all phase lfp

In [12]:
def phaselfp_1file_extract(file, cols):
    """
        return all the phase lfp data from all the trials in one file
    """
    
    matdat = sio.loadmat(file, variable_names = ['lfpdata', 'idxevent_Matrix', 'fs'], 
                         struct_as_record = False, squeeze_me = True)

    # fs: sample rate
    fs = matdat['fs'] 

    # idxevent (n_trials * 5): indices for targetonset, reachonset, reach, return and mouth
    idxevent = matdat['idxevent_Matrix']
        
    # lfp: nchns * ntemp * ntrials
    lfp = matdat['lfpdata']
    
     # idxdur_min, idxdur_max
    idxdur_min, idxdur_max = int(np.round(tdur_min * fs)), int(np.round(tdur_max * fs))
    
    if(len(lfp.shape) ==2 or len(idxevent.shape) == 1):
        # only has one trial 
        
        if(len(lfp.shape) ==2):
            lfp = np.expand_dims(lfp, axis = 2)
            
        if(len(idxevent.shape) == 1):
            idxevent = np.expand_dims(idxevent, axis = 0)
        
    
    ntrials = idxevent.shape[0]
    for triali in range(ntrials):
        
        # start and end indices over time of specific phase for triali 
        idxtime_str, idxtime_end = int(idxevent[triali, cols[0]]), int(idxevent[triali, cols[1]])

        # remove trials of return phase longer than tdur_max and shorter than tdur_min
        if idxtime_end - idxtime_str < idxdur_min or idxtime_end - idxtime_str > idxdur_max:
            continue;
            
            
        # extract lfp data in specific phase of one trial (n_chns * n_times)
        phaselfp_1trial = lfp[:, idxtime_str:idxtime_str + idxdur_min , triali]
        phaselfp_1trial = np.expand_dims(phaselfp_1trial, axis = 2)
        
        if 'phaselfp_alltrials'  not in locals():
            phaselfp_alltrials = phaselfp_1trial
        else:
            phaselfp_alltrials = np.concatenate((phaselfp_alltrials, phaselfp_1trial), axis = 2)
    
    
    if 'phaselfp_alltrials'  not in locals():
        phaselfp_alltrials = []
    
    return phaselfp_alltrials

In [13]:
files = glob.glob(os.path.join(inputfolder, '*.mat'))

# delete phaselfp_normal and phaselfp_mild if exist inside this function
if 'phaselfp_normal' in locals():
    del phaselfp_normal

if 'phaselfp_mild' in locals():
    del phaselfp_mild
    
for file in files:
    
    phaselfp_lfile = phaselfp_1file_extract(file, cols);
    
    # normal case
    if 'normal' in file and 'mild' not in file:
        if 'phaselfp_normal'  not in locals():
            phaselfp_normal = phaselfp_lfile
        else:
            phaselfp_normal = np.concatenate((phaselfp_normal, phaselfp_lfile), axis = 2)
        
        
        
    # mild case
    if 'mild' in file and 'normal' not in file:
        if 'phaselfp_mild'  not in locals():
            phaselfp_mild = phaselfp_lfile
        else:
            phaselfp_mild = np.concatenate((phaselfp_mild, phaselfp_lfile), axis = 2)
            
            
    del phaselfp_lfile

### balance mild and normal trials number

In [14]:
if phaselfp_mild.shape[2] < phaselfp_normal.shape[2]:
    ntrials_ba = phaselfp_mild.shape[2]
else:
    ntrials_ba = phaselfp_normal.shape[2]

idx_batrials_normal = random.sample(range(0, phaselfp_normal.shape[2]), ntrials_ba) 
idx_batrials_mild = random.sample(range(0, phaselfp_mild.shape[2]), ntrials_ba) 


# balance trials
phaselfp_normal_ba = phaselfp_normal[:,:,idx_batrials_normal]
phaselfp_mild_ba = phaselfp_mild[:,:,idx_batrials_mild]

### Bipolar DBS

In [16]:
# STN channel numbers
ch1_STN, ch2_STN = leadSTN_chns[0] + 128, leadSTN_chns[1] + 128

# get the bipolar lfp by substrating 
lfpSTN_normal = np.expand_dims(phaselfp_normal_ba[ch1_STN, :, :] - phaselfp_normal_ba[ch2_STN, :, :], axis = 0)
lfpSTN_mild = np.expand_dims(phaselfp_mild_ba[ch1_STN, :, :] - phaselfp_mild_ba[ch2_STN, :, :], axis = 0)

In [17]:
# GP channel numbers
ch1_GP, ch2_GP = leadGP_chns[0] + 128, leadGP_chns[1] + 128

# get the bipolar lfp by substrating 
lfpGP_normal = np.expand_dims(phaselfp_normal_ba[ch1_GP, :, :] - phaselfp_normal_ba[ch2_GP, :, :], axis = 0)
lfpGP_mild = np.expand_dims(phaselfp_mild_ba[ch1_GP, :, :] - phaselfp_mild_ba[ch2_GP, :, :], axis = 0)

### channel information

In [18]:
### load channel information ###
df = pd.read_csv(chnInf_file, header = 0)


###  channel matchni and recording chni  ###
chnInf_matchni, chnInf_recchni = df['mat_chni'], df['recording_chni']

chnInf_brainarea = df['brainarea']

x, y = df['simulated_x'].to_numpy(), df['simulated_y'].to_numpy()
chnInf_coord = np.concatenate((np.expand_dims(x, axis = 1), np.expand_dims(y, axis = 1)), axis = 1)



### Remove nonused channels ###

# extract the indices in which the corresponding channels are not used
idx_mask = ~chnInf_brainarea.isnull()
idx_notused = [i for i, isnan in enumerate(idx_mask) if isnan == False]


chnInf_recchni = chnInf_recchni[idx_mask].to_numpy()
chnInf_matchni = chnInf_matchni[idx_mask].to_numpy()
chnInf_brainarea = chnInf_brainarea[idx_mask].tolist()
chnInf_coord = chnInf_coord[idx_mask]

### remove the data of the noused channels

In [19]:
if phaselfp_normal_ba.shape[0] == phaselfp_mild_ba.shape[0]:
    nchns = phaselfp_normal_ba.shape[0]


# # extract the idx whose channels are not used
matchni_notused = np.arange(0, nchns, 1).tolist()

for idx, matchni in enumerate(chnInf_matchni):
    if 'STN' in chnInf_brainarea[idx] or 'GP' in chnInf_brainarea[idx]:
        continue
        
    matchni_notused.remove(matchni)

In [20]:
print(phaselfp_mild_ba.shape)

# remove the data of the channel not used
phaselfp_normal_ba = np.delete(phaselfp_normal_ba, matchni_notused, axis = 0)

# remove the data of the channel not used
phaselfp_mild_ba = np.delete(phaselfp_mild_ba, matchni_notused, axis = 0)

print(phaselfp_mild_ba.shape)

# add bipolar STN and GP lfp data
phaselfp_normal_ba = np.concatenate((phaselfp_normal_ba, lfpSTN_normal, lfpGP_normal), axis = 0)
phaselfp_mild_ba = np.concatenate((phaselfp_mild_ba, lfpSTN_mild, lfpGP_mild), axis = 0)

print(phaselfp_mild_ba.shape)

(123, 250, 50)
(125, 250, 50)


## Calc ciCOH

In [29]:
def calciCOH_lfp(lfp):
    
    """
        return the ciCOH extracted from lfp data
        
        Args:
        
            lfp: local field potential data (nchns * ntemporals * ntrials)
            
        Return:
            ciCOH_alltrials: ciCOH values for all trials extracted from lfp data (nchns * nchns * ntrials)
    """
 
    
    ### calculate ciCOH between each pair of channels ###
    
    nchns, ntemp, ntrials = lfp.shape
    
    ciCOH_alltrials = np.zeros((nchns, nchns, ntemp))
    for chni in range(nchns-1):

        # signal of channel chni
        signal1 = lfp[chni, :, :]
        signal1 = np.transpose(signal1) 

        for chnj in range(chni+1, nchns):

            # signal of channel chnij
            signal2 = lfp[chnj, :, :]
            signal2 = np.transpose(signal2)

            # ciCOH_alltrials assignment
            ciCOH_alltrials[chni, chnj, :] =  ciCoherence_acrosstrials(signal1, signal2)

            # symmetrical
            ciCOH_alltrials[chnj, chni, :] = ciCOH_alltrials[chni, chnj, :]

            del signal2
        del signal1


    # return
    return ciCOH_alltrials

In [33]:
ciCOH_temp_normal = calciCOH_lfp(phaselfp_normal_ba)
ciCOH_temp_mild = calciCOH_lfp(phaselfp_mild_ba)

In [35]:
ciCOH_normal, ciCOH_mild = np.mean(ciCOH_temp_normal,axis = 2), np.mean(ciCOH_temp_mild, 2)

/Users/linglingyang/miniconda3/envs/general/lib/python3.6/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


## Save

In [36]:
ciCOH = dict()
ciCOH['normal'], ciCOH['mild'] = ciCOH_normal, ciCOH_mild
ciCOH['chnInf_recchni'], ciCOH['chnInf_matchni'] = chnInf_recchni, chnInf_matchni
ciCOH['chnInf_brainarea'], ciCOH['chnInf_coord'] = chnInf_brainarea, chnInf_coord

In [2]:
with open(os.path.join(savefolder, savefilename), 'wb') as fp:
    pickle.dump(ciCOH, fp, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'savefolder' is not defined